<a href="https://colab.research.google.com/github/AndroBuddy/cricket-data-set/blob/master/Score_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/AndroBuddy/cricket-data-set.git

fatal: destination path 'cricket-data-set' already exists and is not an empty directory.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
# from sklearn.metrics import confusion_matrix, classification_report

In [3]:
import pandas as pd
import glob

path = r'/content/cricket-data-set' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    matches = pd.read_csv(filename, index_col=None, header=0)
    matches["total_runs"] = matches.runs_off_bat + matches.extras
    column = pd.unique(matches["batting_team"])
    df = matches.groupby(matches["batting_team"])
    try:
      df1 = df.get_group(column[0])
      df2 = df.get_group(column[1])

      df1["cumulative"] = df1["total_runs"].cumsum()
      df2["cumulative"] = df2["total_runs"].cumsum()

      matches = pd.concat([df1,df2])
      li.append(matches)
    except:
      print(filename)
      print("HELOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO") ## one of the files : 501265 has some error
      continue

matches = pd.concat(li, axis=0, ignore_index=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


/content/cricket-data-set/501265.csv
HELOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO


In [4]:
matches_copy = matches.copy()

In [5]:
matches.season.iloc[0]

2012

In [6]:
matches  = matches.drop(columns = ["season" , "match_id" , "start_date" , "wicket_type" , "player_dismissed" , "other_wicket_type" , "other_player_dismissed"])

In [7]:
matches  = matches.drop(columns = ["runs_off_bat" , "extras" , "wides" , "noballs" , "byes" , "legbyes" , "penalty"])

In [8]:
matches.columns

Index(['venue', 'innings', 'ball', 'batting_team', 'bowling_team', 'striker',
       'non_striker', 'bowler', 'total_runs', 'cumulative'],
      dtype='object')

In [9]:
pd.unique(matches["venue"])

array(['MA Chidambaram Stadium, Chepauk', 'Sawai Mansingh Stadium',
       'M Chinnaswamy Stadium', 'Eden Gardens',
       'Dr DY Patil Sports Academy', 'New Wanderers Stadium',
       'Brabourne Stadium', 'Subrata Roy Sahara Stadium',
       'Punjab Cricket Association Stadium, Mohali', 'Kingsmead',
       'SuperSport Park', 'Himachal Pradesh Cricket Association Stadium',
       'Rajiv Gandhi International Stadium, Uppal', 'Feroz Shah Kotla',
       'Newlands', 'Wankhede Stadium', 'De Beers Diamond Oval',
       'Barabati Stadium', 'Nehru Stadium', 'Buffalo Park',
       'Sardar Patel Stadium, Motera', 'Holkar Cricket Stadium',
       'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium',
       'Vidarbha Cricket Association Stadium, Jamtha', "St George's Park",
       'OUTsurance Oval'], dtype=object)

In [10]:
current_teams = ['Kolkata Knight Riders', 'Chennai Super Kings', 'Rajasthan Royals', 'Mumbai Indians', 'Kings XI Punjab', 'Royal Challengers Bangalore', 'Delhi Daredevils', 'Sunrisers Hyderabad']

In [11]:
matches = matches[(matches['batting_team'].isin(current_teams)) & (matches['bowling_team'].isin(current_teams))]

In [12]:
pd.unique(matches["batting_team"])

array(['Chennai Super Kings', 'Kolkata Knight Riders', 'Rajasthan Royals',
       'Mumbai Indians', 'Royal Challengers Bangalore',
       'Delhi Daredevils', 'Kings XI Punjab'], dtype=object)

In [13]:
matches.columns

Index(['venue', 'innings', 'ball', 'batting_team', 'bowling_team', 'striker',
       'non_striker', 'bowler', 'total_runs', 'cumulative'],
      dtype='object')

In [14]:
venue_encoder = LabelEncoder()
matches["venue"] = venue_encoder.fit_transform(matches["venue"])
batting_team_encoder = LabelEncoder()
matches["batting_team"] = batting_team_encoder.fit_transform(matches["batting_team"])
bowling_team_encoder = LabelEncoder()
matches["bowling_team"] = bowling_team_encoder.fit_transform(matches["bowling_team"])
striker_encoder = LabelEncoder()
matches["striker"] = striker_encoder.fit_transform(matches["striker"])
non_striker_encoder = LabelEncoder()
matches["non_striker"] = non_striker_encoder.fit_transform(matches["non_striker"])
bowler_encoder = LabelEncoder()
matches["bowler"] = bowler_encoder.fit_transform(matches["bowler"])

In [15]:
matches.columns

Index(['venue', 'innings', 'ball', 'batting_team', 'bowling_team', 'striker',
       'non_striker', 'bowler', 'total_runs', 'cumulative'],
      dtype='object')

In [25]:
# features = ["venue" , "innings" , "ball" , "batting_team" , "bowling_team" , "striker" , "non_striker" , "bowler" , "total_runs" , "cumulative"]
features = ["venue" , "innings" , "ball" , "batting_team" , "bowling_team" , "striker" , "non_striker" , "bowler"]

# features = [ "innings" , "ball" , "total_runs"]

In [26]:
matches_copy['count'] = 1
matches_copy.groupby(['venue']).count()['count']

venue
Barabati Stadium                                        973
Brabourne Stadium                                      1720
Buffalo Park                                            715
De Beers Diamond Oval                                   726
Dr DY Patil Sports Academy                             3993
Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium     476
Eden Gardens                                           6451
Feroz Shah Kotla                                       6160
Himachal Pradesh Cricket Association Stadium           1673
Holkar Cricket Stadium                                  408
Kingsmead                                              3643
M Chinnaswamy Stadium                                  6398
MA Chidambaram Stadium, Chepauk                        8050
Nehru Stadium                                          1155
New Wanderers Stadium                                  1940
Newlands                                               1539
OUTsurance Oval                   

In [27]:
matches = matches[matches["ball"] < 6]

In [ ]:
matches.head(38)

In [28]:
x_train = matches[features].head(12380)
x_test = matches[features].tail(matches.shape[0] - 12380)

In [29]:
y_train = matches[features].head(12380)
y_test = matches[features].tail(matches.shape[0] - 12380)

12380.8